In [1]:
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
prosody_data = pd.read_csv('Prosodic_Features_MRH.csv',sep=",")
#expr = pd.read_csv(expr_file, sep="\t")
y = prosody_data['Hired3C'].values
#36
#X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD']]
#49
X =prosody_data[['duration', 'energy', 'power', 'min_pitch', 'max_pitch', 'mean_pitch', 'pitch_sd', 'pitch_abs', 'pitch_quant', 'pitchUvsVRatio', 'Time:8', 'iDifference', 'diffPitchMaxMin', 'diffPitchMaxMean', 'diffPitchMaxMode', 'intensityMin', 'intensityMax', 'intensityMean', 'intensitySD', 'intensityQuant', 'diffIntMaxMin', 'diffIntMaxMean', 'diffIntMaxMode', 'avgVal1', 'avgVal2', 'avgVal3', 'avgBand1', 'avgBand2', 'avgBand3', 'fmean1', 'fmean2', 'fmean3', 'f2meanf1', 'f3meanf1', 'f1STD', 'f2STD', 'f3STD', 'f2STDf1', 'f2STDf2', 'jitter', 'shimmer', 'jitterRap', 'meanPeriod', 'percentUnvoiced', 'numVoiceBreaks', 'PercentBreaks', 'speakRate', 'numPause', 'maxDurPause']]
#25X = prosody_data[['pitch_quant', 'meanPeriod', 'mean_pitch', 'fmean3', 'f3STD', 'f2STD', 'pitch_sd', 'shimmer', 'avgBand2', 'intensityMean', 'jitter', 'intensitySD', 'intensityQuant', 'jitterRap', 'f1STD', 'avgVal2', 'fmean2', 'PercentBreaks', 'AvgTotFall:3', 'f2STDf1', 'pitch_abs', 'f2STDf2', 'avgBand1', 'percentUnvoiced', 'AvgTotRis:3']]
#X = prosody_data.iloc[:, 4:].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=23, stratify=y)
X_train.shape

(612, 49)

In [3]:
#parallel cocordinate graph
 

ln = LogScaler()
X_train_norm = ln.fit_transform(X_train)
X_test_norm = ln.transform(X_test)
X_train_norm

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_norm =  scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
X_train_norm

In [4]:
imDict={}
print(len(prosody_data))
dim=(7,7)
num_classes = 3
x_imageSet=np.empty((len(prosody_data),dim[0],dim[1]))

y_Set=np.empty((len(prosody_data),1))



680


In [5]:
for index, row in X.iterrows():
    mat = np.reshape(row.to_numpy(),(7,7))
    x_imageSet[index]=mat
    y_Set[index] = y[index]
    imDict[index]=(x_imageSet[index],y_Set[index])
   


imDict

In [6]:
def load_data(imDict):
    train_len = int(len(prosody_data)*0.8)
    test_len =  len(prosody_data) - train_len
    x_trainImSet=np.empty((train_len,dim[0],dim[1]))
    x_testImSet=np.empty(( test_len,dim[0],dim[1]))
    y_trainSet=np.zeros(train_len)
    y_testSet=np.zeros(test_len)
    for trnPi in range(train_len):
        (x_trainImSet[trnPi],y_trainSet[trnPi])=imDict[trnPi]
    
    for testPi in range(test_len):
        (x_testImSet[testPi],y_testSet[testPi])=imDict[testPi]
    
    x_trainImSet= x_trainImSet.astype('float32')
    x_testImSet= x_testImSet.astype('float32')
   
    
    # convert class vectors to matrices as binary
    
    y_trainSet= tensorflow.keras.utils.to_categorical(y_trainSet, num_classes)
    y_testSet= tensorflow.keras.utils.to_categorical(y_testSet, num_classes)
    
    return (x_trainImSet,y_trainSet), (x_testImSet,y_testSet)

In [7]:
(X_train,y_train) , (X_test,y_test) = load_data(imDict)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((544, 7, 7), (544, 3), (136, 7, 7), (136, 3))

In [8]:
train_X = X_train.reshape(544, 7, 7,1)
valid_X = X_test.reshape(136, 7, 7,1)
train_label = y_train.reshape(544,3)
valid_label = y_test.reshape(136,3)

In [9]:
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((544, 7, 7, 1), (136, 7, 7, 1), (544, 3), (136, 3))

In [10]:
batch_size = 64
epochs = 50
num_classes = 3

In [11]:


import datetime
start = datetime.datetime.now()

fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(7,7,1),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(Dense(num_classes, activation='softmax'))

fashion_model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(),metrics=['accuracy'])
fashion_model.summary()

fashion_train = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
(loss, accuracy) = fashion_model.evaluate( 
    valid_X, valid_label, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100)) 
print("[INFO] Loss: {}".format(loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 7, 32)          320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 7, 7, 32)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 64)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         7

Epoch 45/50
9/9 [==============================] - 0s 15ms/step - loss: 0.9385 - accuracy: 0.5533 - val_loss: 1.2949 - val_accuracy: 0.4044
Epoch 46/50
9/9 [==============================] - 0s 24ms/step - loss: 1.0562 - accuracy: 0.5037 - val_loss: 1.6987 - val_accuracy: 0.4044
Epoch 47/50
9/9 [==============================] - 0s 22ms/step - loss: 1.2932 - accuracy: 0.4449 - val_loss: 1.3388 - val_accuracy: 0.3824
Epoch 48/50
9/9 [==============================] - 0s 20ms/step - loss: 1.0215 - accuracy: 0.5165 - val_loss: 1.1761 - val_accuracy: 0.4191
Epoch 49/50
9/9 [==============================] - 0s 20ms/step - loss: 0.9719 - accuracy: 0.5404 - val_loss: 1.0943 - val_accuracy: 0.4632
Epoch 50/50
3/3 [==============================] - 0s 5ms/step - loss: 1.0864 - accuracy: 0.4632
[INFO] accuracy: 46.32%
[INFO] Loss: 1.0864384174346924
Time:  0:00:09.341600
